<a href="https://colab.research.google.com/github/Nik8x/TF_Speech_Recognition/blob/master/TF_Speech_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import glob
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline
import IPython

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, Conv2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

In [0]:
# let's see total audio files for each label
label = []
counts = []
for i in os.listdir('../input/tensorflow-speech-recognition-challenge/train/audio/'):
    label.append(i)
    counts.append(len(os.listdir('../input/tensorflow-speech-recognition-challenge/train/audio/' + i)))
    
df = pd.DataFrame([label, counts]).T.set_index([0])
df

,1
0,
right,2367
tree,1733
down,2359
go,2372
wow,1745
bed,1713
zero,2376
stop,2380
off,2357


In [0]:
# Total number of audio files
np.sum(df[1])

64728

In [0]:
os.listdir('../input/tensorflow-speech-recognition-challenge/')

['train', 'sample_submission.csv']

In [0]:
test_list = open('../input/tensorflow-speech-recognition-challenge/train/testing_list.txt').read().split('\n')
val_list = open('../input/tensorflow-speech-recognition-challenge/train/validation_list.txt').read().split('\n')

In [0]:
len(test_list), len(val_list)

(6836, 6799)

In [0]:
IPython.display.Audio('../input/tensorflow-speech-recognition-challenge/train/audio/right/bf8d5617_nohash_0.wav')

In [0]:
labels_in_test = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
df = df.loc[labels_in_test]
df

In [0]:
np.sum(df[1])

23682

In [0]:
%%time
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

def extract_data(folder):
    dff = pd.DataFrame()
    names = []
    features = []
    
    for name in os.listdir('../input/tensorflow-speech-recognition-challenge/train/audio/' + folder):
        data, sampling_rate = librosa.load('../input/tensorflow-speech-recognition-challenge/train/audio/' + folder + '/' + name, res_type = 'kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y = data, sr = sampling_rate, n_mfcc = 40).T, axis = 0)
        
        features.append(mfccs)
        names.append(name)
        
    return [names, features]

train_new = pd.Series(labels_in_test).apply(extract_data)
train_new.columns = ['names', 'features']

CPU times: user 12min 57s, sys: 16min 4s, total: 29min 2s
Wall time: 7min 22s


In [0]:
dfff = pd.DataFrame()
dfff['names'] = train_new[0][0] + train_new[1][0] + train_new[2][0] + train_new[3][0] + train_new[4][0] + train_new[5][0] + train_new[6][0] + train_new[7][0] + train_new[8][0] + train_new[9][0]
dfff['feature'] = train_new[0][1] + train_new[1][1] + train_new[2][1] + train_new[3][1] + train_new[4][1] + train_new[5][1] + train_new[6][1] + train_new[7][1] + train_new[8][1] + train_new[9][1]

In [0]:
label_all = []
for index in labels_in_test:
    label_all.append([index] * df.loc[index][1])

In [0]:
dfff['label'] = [item for sublist in label_all for item in sublist]

In [0]:
print(dfff.shape)
dfff.head()

(23682, 3)


,names,feature,label
0,5fe4a278_nohash_1.wav,"[-360.960391618762, 95.24125277596205, -45.965...",yes
1,4def68db_nohash_1.wav,"[-350.56030750703735, 95.85476962473145, -18.2...",yes
2,3d53244b_nohash_3.wav,"[-452.9357667760163, 48.59143620106703, -8.356...",yes
3,62ef962d_nohash_1.wav,"[-304.8770846153024, 60.81771523850327, -32.59...",yes
4,264f471d_nohash_3.wav,"[-305.4094163819084, 100.41015855982204, -2.31...",yes


In [0]:
dfff.to_csv("dfff.csv", index = False, sep = ',')

In [0]:
val_list_all = pd.Series(val_list[:-1]).apply(lambda x: x.split('/')[1])
test_list_all = pd.Series(test_list[:-1]).apply(lambda x: x.split('/')[1])
len(val_list_all), len(test_list_all)

(6798, 6835)

In [0]:
val = dfff[dfff.names.isin(val_list_all)]
val = val.iloc[np.random.permutation(len(val))]  # Shuffle DataFrame rows
print(val.shape)

train = dfff[~dfff.names.isin(val_list_all)]
train = train.iloc[np.random.permutation(len(train))] # Shuffle DataFrame rows
print(train.shape)

(2577, 3)
(21105, 3)


In [0]:
train.head()

,names,feature,label
4466,33f60c62_nohash_0.wav,"[-666.1384645460589, 105.0935253333153, -36.90...",no
14015,8e05039f_nohash_0.wav,"[-408.13521014083494, 112.73906017658597, 0.49...",right
23485,f875f965_nohash_0.wav,"[-470.8189676086632, 143.73396008009183, -24.7...",go
8363,3e31dffe_nohash_3.wav,"[-454.153372075609, 162.1608907677855, -12.808...",down
19479,8134f43f_nohash_2.wav,"[-416.61373092197425, 147.70438559348352, 28.4...",stop


In [0]:
X_train = np.array(train.feature.tolist())
y_train = np.array(train.label.tolist())

X_val = np.array(val.feature.tolist())
y_val = np.array(val.label.tolist())

lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_val = np_utils.to_categorical(lb.fit_transform(y_val))

In [0]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((21105, 40), (21105, 10), (2577, 40), (2577, 10))

In [0]:
ncols = X_train.shape[1]
model = Sequential()

model.add(Dense(1024, activation = 'relu', input_shape = (ncols,)))
model.add(Dropout(0.5))

model.add(Dense(512, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(y_train.shape[1], activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
%%time
model.fit(X_train, y_train, batch_size = 32, epochs = 10, validation_data = [X_val, y_val])

Train on 21105 samples, validate on 2577 samples
Epoch 1/10
21105/21105 [==============================] - 9s 406us/step - loss: 14.4880 - acc: 0.0983 - val_loss: 14.4669 - val_acc: 0.1024
Epoch 2/10
21105/21105 [==============================] - 8s 397us/step - loss: 14.4947 - acc: 0.1005 - val_loss: 14.4669 - val_acc: 0.1024
Epoch 3/10
21105/21105 [==============================] - 7s 337us/step - loss: 14.5198 - acc: 0.0991 - val_loss: 14.4669 - val_acc: 0.1024
Epoch 4/10
21105/21105 [==============================] - 7s 327us/step - loss: 14.5174 - acc: 0.0993 - val_loss: 14.4669 - val_acc: 0.1024
Epoch 5/10
21105/21105 [==============================] - 7s 328us/step - loss: 14.5174 - acc: 0.0993 - val_loss: 14.4669 - val_acc: 0.1024
Epoch 6/10
21105/21105 [==============================] - 7s 329us/step - loss: 14.5181 - acc: 0.0993 - val_loss: 14.4669 - val_acc: 0.1024
Epoch 7/10
21105/21105 [==============================] - 7s 330us/step - loss: 14.5181 - acc: 0.0993 - val_los

In [0]:
X_train.reshape(X_train.shape[0], 8, 5, 1).shape

(21105, 8, 5, 1)

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(8, 5, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
%%time
model.fit(X_train.reshape(X_train.shape[0], 8, 5, 1), y_train, batch_size = 32, epochs = 10, validation_data = [X_val.reshape(X_val.shape[0], 8, 5, 1), y_val], verbose = 1)

Train on 21105 samples, validate on 2577 samples
Epoch 1/10
21105/21105 [==============================] - 2s 93us/step - loss: 1.8132 - acc: 0.3497 - val_loss: 1.7022 - val_acc: 0.4296
Epoch 2/10
21105/21105 [==============================] - 2s 80us/step - loss: 1.8085 - acc: 0.3544 - val_loss: 1.7039 - val_acc: 0.4288
Epoch 3/10
21105/21105 [==============================] - 2s 78us/step - loss: 1.8042 - acc: 0.3547 - val_loss: 1.6959 - val_acc: 0.4106
Epoch 4/10
21105/21105 [==============================] - 2s 78us/step - loss: 1.7984 - acc: 0.3548 - val_loss: 1.6864 - val_acc: 0.4133
Epoch 5/10
21105/21105 [==============================] - 2s 77us/step - loss: 1.8019 - acc: 0.3569 - val_loss: 1.6937 - val_acc: 0.4187
Epoch 6/10
21105/21105 [==============================] - 2s 78us/step - loss: 1.7972 - acc: 0.3577 - val_loss: 1.7058 - val_acc: 0.4183
Epoch 7/10
21105/21105 [==============================] - 2s 78us/step - loss: 1.8005 - acc: 0.3575 - val_loss: 1.6840 - val_acc: